# Intentions

DA-VINCIS: is the acronym for **Detection of Aggressive and Violent Incidents from Social Media in Spanish**

> With this project we try to detect the aggresive and violent content, with this work we are trying to be able to prevent identify people that witness or experience violence.

# Imports and Global Variables Definition

In [ ]:
# instalar librerías. Esta casilla es últil por ejemplo si se ejecuta el cuaderno en Google Colab
# Note que existen otras dependencias como tensorflow, etc. que en este caso se encontrarían ya instaladas
#
!pip install autogoal[contrib]==0.4.4
print('Done!')

In [ ]:

#!pip install WordCloud
#!pip install NLTK
#!pip install plotly

In [ ]:

#  para construir gráficas y realizar análisis exploratorio de los datos
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

# para cargar datos y realizar pre-procesamiento básico
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder


# para evaluar los modelos 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, f1_score
from sklearn.utils.multiclass import unique_labels

# para configurar AutoGOAL
from autogoal.ml import AutoML
from autogoal.search import (Logger, PESearch, ConsoleLogger, ProgressLogger, MemoryLogger)
from autogoal.kb import MatrixContinuousDense, VectorCategorical, Supervised
from autogoal.contrib import find_classes

# para guardar el modelo
import pickle
import datetime

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words_sp = stopwords.words('spanish')

#### CARLOS OSMAN SUAZO BACA #####
# Cargamos esta librería para revisar el desbalance que existe en el dataset de tripadvisor
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler
# Para contar los parámetros de las clases 
from collections import Counter

print(type(stop_words_sp))
print(stop_words_sp)
print('Done!')

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# para construir pipelines
from sklearn.pipeline import Pipeline

# para evaluar los modelos 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import StandardScaler


# algoritmos de clasificación
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

TEXT_COL      = 'tweet'
CLASS_COL_ST1 = 'Sentiment'
CLASS_COL_ST2 = ['Theft', 'Homicide', 'Kidnapping', 'Accident', 'None of the above']

# Declaramos algunas variables globales
N_JOBS = 6 # Número de núclos a implementar por gridsearch para el hyper parámeter tuning
CV = 5 # Número de interaciones para hacer cross validation.

# Auxiliar Functions

In [ ]:
# función auxiliar utilizada por CountVectorizer para procesar las frases
def spanish_stemmer(sentence):
    stemmer = SpanishStemmer()
    analyzer = CountVectorizer(binary=False, analyzer='word', stop_words=stop_words_sp,
                               ngram_range=(1, 1)).build_analyzer()
    return (stemmer.stem(word) for word in analyzer(sentence))


# guarda un pipeline entrenado
def save_model(model, modelName = "models/pickle_model.pkl"):
   pkl_filename = modelName
   with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)   


# carga un pipeline entrenado y guardado previamente
def load_model(rutaModelo = "models/pickle_model.pkl"):
  # Load from file
  with open(rutaModelo, 'rb') as file:
    pickle_model = pickle.load(file)
    return pickle_model 


# función auxiliar para realizar predicciones con el modelo
def predict_model(model, data, pref='m'):
  """
  data: list of the text to predict
  pref: identificador para las columnas (labels_[pref], scores_[pref]_[class 1], etc.)
  """
  res = {}
  scores = None
  labels = model.predict(data)

  if hasattr(model, 'predict_proba'):
    scores = model.predict_proba(data)
  
    # empaquetar scores dentro de un diccionario que contiene labels, scores clase 1, scores clase 2, .... El nombre de la clase se normaliza a lowercase
    #res = {f'scores_{pref}_{cls.lower()}':score for cls, score in zip(model.classes_, [col for col in scores.T])}
    res = {f'scores_{pref}_{cls}':score for cls, score in zip(model.classes_, [col for col in scores.T])}

  # añadir datos relativos a la predicción
  res[f'labels_{pref}'] = labels

  # convertir a dataframe ordenando las columnas primero el label y luego los scores por clase, las clases ordenadas alfabeticamente.
  res = pd.DataFrame(res, columns=sorted(list(res.keys())))

  return res


# función auxiliar que evalúa los resultados de una clasificación
def evaluate_model(y_true, y_pred, y_score=None, pos_label='positive'):
  """
  
  """
  print('==== Sumario de la clasificación ==== ')
  print(classification_report(y_true, y_pred))

  print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))

  # graficar matriz de confusión
  display_labels = sorted(unique_labels(y_true, y_pred), reverse=True)
  cm = confusion_matrix(y_true, y_pred, labels=display_labels)

  z = cm[::-1]
  x = display_labels
  y =  x[::-1].copy()
  z_text = [[str(y) for y in x] for x in z]

  fig_cm = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

  fig_cm.update_layout(
      height=400, width=400,
      showlegend=True,
      margin={'t':150, 'l':0},
      title={'text' : 'Matriz de Confusión', 'x':0.5, 'xanchor': 'center'},
      xaxis = {'title_text':'Valor Real', 'tickangle':45, 'side':'top'},
      yaxis = {'title_text':'Valor Predicho', 'tickmode':'linear'},
  )
  fig_cm.show()


# función auxiliar para realizar predicciones con el modelo
def _predict_model(model, cfg, data, pref='m'):
  """
  data: list of the text to predict
  pref: identificador para las columnas (labels_[pref], scores_[pref]_[class 1], etc.)
  """
  res = {}
  scores = None

  data_tfidf = cfg['vectorizer'].transform(data)
  data_tfidf = data_tfidf.todense()
  labels = model.predict(data_tfidf)

  if hasattr(model, 'predict_proba'):
    scores = model.predict_proba(data_tfidf)
  
    # empaquetar scores dentro de un diccionario que contiene labels, scores clase 1, scores clase 2, .... El nombre de la clase se normaliza a lowercase
    res = {f'scores_{pref}_{cls.lower()}':score for cls, score in zip(model.classes_, [col for col in scores.T])}

  # añadir datos relativos a la predicción
  res[f'labels_{pref}'] = cfg['label_encoder'].inverse_transform(labels)

  # convertir a dataframe ordenando las columnas primero el label y luego los scores por clase, las clases ordenadas alfabeticamente.
  res = pd.DataFrame(res, columns=sorted(list(res.keys())))

  return res


# función auxiliar que evalúa los resultados de una clasificación
def _evaluate_model(y_true, y_pred, y_score=None, pos_label='positive'):
  """
  data: list of the text to predict
  pref: identificador para las columnas (labels_[pref], scores_[pref]_[class 1], etc.)
  """
  print('==== Sumario de la clasificación ==== ')
  print(classification_report(y_true, y_pred))

  print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))

  # graficar matriz de confusión
  display_labels = sorted(unique_labels(y_true, y_pred), reverse=True)
  cm = confusion_matrix(y_true, y_pred, labels=display_labels)

  z = cm[::-1]
  x = display_labels
  y =  x[::-1].copy()
  z_text = [[str(y) for y in x] for x in z]

  fig_cm = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

  fig_cm.update_layout(
      height=400, width=400,
      showlegend=True,
      margin={'t':150, 'l':0},
      title={'text' : 'Matriz de Confusión', 'x':0.5, 'xanchor': 'center'},
      xaxis = {'title_text':'Valor Real', 'tickangle':45, 'side':'top'},
      yaxis = {'title_text':'Valor Predicho', 'tickmode':'linear'},
  )
  fig_cm.show()


  # curva roc (definido para clasificación binaria)
  fig_roc = None
  if y_score is not None:
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=pos_label)
    fig_roc = px.area(
        x=fpr, y=tpr,
        title = f'Curva ROC (AUC={auc(fpr, tpr):.4f})',
        labels=dict(x='Ratio Falsos Positivos', y='Ratio Verdaderos Positivos'),
        width=400, height=400
    )
    fig_roc.add_shape(type='line', line=dict(dash='dash'), x0=0, x1=1, y0=0, y1=1)

    fig_roc.update_yaxes(scaleanchor="x", scaleratio=1)
    fig_roc.update_xaxes(constrain='domain')
    
    fig_roc.show()

# Read Data

In [ ]:
# leemos el corpus de tweets y los labels correspondientes a 
# las tareas 1 de clasificación binaria y clasificación multilabel
df_train            = pd.read_csv('https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2022/main/data/training_data/train_data.csv', header=None, names = [TEXT_COL])
df_train_labels_st1 = pd.read_csv('https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2022/main/data/training_data/train_labels_subtask_1.csv', header=None, names = [CLASS_COL_ST1])
df_train_labels_st2 = pd.read_csv('https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2022/main/data/training_data/train_labels_subtask_2.csv', header=None, names = CLASS_COL_ST2)

In [ ]:

df_train_lst1 = pd.concat([df_train, df_train_labels_st1], axis = 1)
df_train_lst2 = pd.concat([df_train, df_train_labels_st2], axis = 1)
display(df_train_lst1)
display(df_train_lst2)

In [ ]:
# obtener algunas estadísticas sobre los datos
categories = sorted(df_train_lst1[CLASS_COL_ST1].unique(), reverse=False)
hist= Counter(df_train_lst1[CLASS_COL_ST1]) 
print(f'Total de instancias -> {df_train_lst1.shape[0]}')

print(f'Categorías -> {categories}')
print(f'Comentario de ejemplo -> {df_train_lst1[CLASS_COL_ST1][0]}')
print(f'Categoría del comentario -> {df_train_lst1[CLASS_COL_ST1][0]}')

fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()
# Hacer entrenamiento con el corpus desbalanceado
# Probar rellenando los datos de las categorías 0 y 1, cortar los datos de la categoría 2 utilizar la función iloc, traer 1052 de cada categoría 
print('Done!')

> No notamos un desbalance significativo en las clases, de momento intentaremos trabajar con estos datos.

In [ ]:
# obtener conjuntos de entrenamiento (90%) y validación (10%)
seed = 0  # fijar random_state para reproducibilidad
train, val = train_test_split(df_train_lst1, test_size=.1, stratify=df_train_lst1[CLASS_COL_ST1], random_state=seed)


In [ ]:
def preprocess_pipeline():
    return Pipeline([
        ('dataVect', CountVectorizer(analyzer=spanish_stemmer)),
        ('tfidf', TfidfTransformer(smooth_idf=True, use_idf=True)),
     ])
    
print('Done!')

# Training SVC

In [ ]:
#### CARLOS OSMAN SUAZO BACA #####
# Aquí entrenaremos el modelo utilizando máquinas de vector soporte para la 
# clasificación.
# crear el pipeline (solo incluyendo los pasos de pre-procesamiento)
model = preprocess_pipeline()

# crear el clasificador y añadirlo al model. Puede probar diferentes clasificadores
classifier = SVC(probability=True)

model.steps.append(('classifier', classifier))

# entrenar el modelo
model.fit(train[TEXT_COL], train[CLASS_COL_ST1])

# guardar el modelo
save_model(model, 'models/svc_model.pkl')

print('Done!')

## Evaluación del modelo SVC
Luego de entrenado el modelo, podemos evaluar su desempeño en los conjuntos de entrenamiento y validación.

Ejecute la siguiente casilla para evaluar el modelo en el conjunto de entrenamiento.

In [ ]:
model = load_model('models/svc_model.pkl')

In [ ]:
# predecir y evaluar el modelo en el conjunto de entrenamiento
print('==== Evaluación conjunto de entrenamiento ====')

true_labels = train[CLASS_COL_ST1]

m_pred = predict_model(model, train[TEXT_COL].to_list(), pref='m')

# el nombre de los campos dependerá de pref al llamar a predic_model y las clases. Ver comentarios en la definición de la función
evaluate_model(true_labels, m_pred['labels_m'], m_pred['scores_m_1'], '1')  

print('Done!')

Ejecutamos la siguiente casilla para evaluar el modelo en el conjunto de validación.

In [ ]:
# predecir y evaluar el modelo en el conjunto de validación
print('==== Evaluación conjunto de validación ====')

true_labels = val[CLASS_COL_ST1]

m_pred = predict_model(model, val[TEXT_COL].to_list(), pref='m')

# el nombre de los campos dependerá de pref al llamar a predic_model y las clases. Ver comentarios en la definición de la función
evaluate_model(true_labels, m_pred['labels_m'], m_pred['scores_m_1'], '1')
# desbalance.
# falta de vopcabulario en las categoría neutral.
# la intermedia es más dificil de clasificar.
print('Done!')

# Training SGDClassifier

In [ ]:
#### CARLOS OSMAN SUAZO BACA #####
# Entrenamos el modelo y los almacenamos para poder hacer las validaciones
# correspondientes. Intentaremos ver que clasificador tiene un mejor score en el 
# conjunto de entrenamiento (train) y el conjunto de validación(test o val).
# crear el pipeline (solo incluyendo los pasos de pre-procesamiento)

model = preprocess_pipeline()

# crear el clasificador y añadirlo al model. Puede probar diferentes clasificadores
classifier = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=1000, tol=None)

model.steps.append(('classifier', classifier))

# entrenar el modelo
model.fit(train[TEXT_COL], train[CLASS_COL_ST1])

# guardar el modelo
save_model(model, 'models/sgd_classifier_model.pkl')

print('Done!')

## Evaluación del modelo SGDClassifier
Luego de entrenado el modelo, podemos evaluar su desempeño en los conjuntos de entrenamiento y validación.

Ejecute la siguiente casilla para evaluar el modelo en el conjunto de entrenamiento.

In [ ]:
model = load_model('models/sgd_classifier_model.pkl')

In [ ]:
# predecir y evaluar el modelo en el conjunto de entrenamiento

print('==== Evaluación conjunto de entrenamiento ====')

print("Precisión: ", model.score(train[TEXT_COL],train[CLASS_COL_ST1]))

print('Done!')

In [ ]:
y_pred = model.predict(train[TEXT_COL])
report = classification_report(train[CLASS_COL_ST1], y_pred, output_dict = True)
report_df = pd.DataFrame(report).transpose()
report_df

Ejecutamos la siguiente casilla para evaluar el modelo en el conjunto de validación.

In [ ]:
# predecir y evaluar el modelo en el conjunto de entrenamiento

print('==== Evaluación conjunto de entrenamiento ====')

print("Precisión: ", model.score(val[TEXT_COL],val[CLASS_COL_ST1]))

print('Done!')

In [ ]:
y_pred = model.predict(val[TEXT_COL])
report = classification_report(val[CLASS_COL_ST1], y_pred, output_dict = True)
report_df = pd.DataFrame(report).transpose()
report_df

# Testing Some Models

In [ ]:
cfg = {}  # diccionario para agrupar configuraciones y variables para su posterior uso

# cargar el LabelEncoder
#with open('models/label_encoder_reviews_test_0.78.pkl', 'rb') as f:
cfg['label_encoder'] = load_model('models/label_encoder_reviews_test_0.78.pkl')

# cargar TfidfVectorizer
#with open('models/vectorizer_reviews_0.78.pkl', 'rb') as f:
cfg['vectorizer'] =load_model('models/vectorizer_reviews_0.78.pkl')

print('Done!')

In [ ]:
# predecir y evaluar el modelo en el conjunto de validación
print('==== Evaluación conjunto de validacióm ====')

true_labels = val[CLASS_COL_ST1]

m_pred = _predict_model(model, cfg, val[TEXT_COL].to_list(), pref='m')

# el nombre de los campos dependerá de pref al llamar a predic_model y las clases. Ver comentarios en la definición de la función
_evaluate_model(true_labels, m_pred['labels_m'])  

print('Done!')

In [ ]:
# from autogoal.ml import AutoML
# from autogoal.datasets import haha
# from autogoal.search import (
#     PESearch,
#     RichLogger,
# )
# from autogoal.kb import Seq, Sentence, VectorCategorical, Supervised
# from autogoal.contrib import find_classes
# from sklearn.metrics import f1_score

# import argparse

# # parser = argparse.ArgumentParser()
# # parser.add_argument("--iterations", type=int, default=10000)
# # parser.add_argument("--timeout", type=int, default=60)
# # parser.add_argument("--memory", type=int, default=2)
# # parser.add_argument("--popsize", type=int, default=50)
# # parser.add_argument("--selection", type=int, default=10)
# # parser.add_argument("--global-timeout", type=int, default=None)
# # parser.add_argument("--examples", type=int, default=None)
# # parser.add_argument("--token", default=None)
# # parser.add_argument("--channel", default=None)

# # args = parser.parse_args()

# # print(args)

# for cls in find_classes():
#     print("Using: %s" % cls.__name__)

# # Load dataset
# X, y = df_train_lst1[TEXT_COL], df_train_lst1[CLASS_COL_ST1]


# X_train = train[TEXT_COL].tolist()
# y_train = train[CLASS_COL_ST1].to_numpy()

# X_test = val[TEXT_COL].tolist()
# y_test = val[CLASS_COL_ST1].to_numpy()

# classifier = AutoML(
#     search_algorithm=PESearch,
#     input=(Seq[Sentence], Supervised[VectorCategorical]),
#     output=VectorCategorical,
#     search_iterations=10000,
#     score_metric = f1_score,
#     errors="warn",
#     pop_size=50,
#     search_timeout= None,
#     evaluation_timeout = 60,
#     memory_limit= 2 * 1024 ** 3,
#     registry=find_classes("Keras|Bert"),
# )

# # loggers = [RichLogger()]

# # if None:
# #     from autogoal.contrib.telegram import TelegramLogger

# #     telegram = TelegramLogger(token=None, name=f"HAHA", channel=None,)
# #     loggers.append(telegram)

# classifier.fit(X_train, y_train)
# score = classifier.score(X_test, y_test)

# print(score)
# print(classifier)